![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)

# <center> Deep Learning Methods </center>
## <center> Lecture 07 - PyTorch</center>
### <center> PyTorch Basics</center>

Colab users should use GPU runtime:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/DeepLearningMethods/07_PyTorch/MainPytorchBasics1.ipynb)

### Useful PyTorch tutorials:
https://pytorch.org/tutorials/

In [1]:
#-- Wide screen:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
#-- Auto reload:
%load_ext autoreload
%autoreload 2

In [3]:
#-- Imports:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rc('font', **{'size' : 16})

#-- torch:
import torch

### Tensors
Tensors are similar to NumPy’s ndarrays,  
Tensors can also be used on GPUs.

In [4]:
mX = torch.ones(2, 3)
mX

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [5]:
#-- type, shape & size:
mX.type(), mX.shape, mX.size()

('torch.FloatTensor', torch.Size([2, 3]), torch.Size([2, 3]))

In [6]:
#-- dytpe = int:
mX = torch.ones(2, 3, dtype=int)
mX, mX.type()

(tensor([[1, 1, 1],
         [1, 1, 1]]),
 'torch.LongTensor')

In [7]:
#-- To NumPy:
vX = torch.linspace(1, 3, 15)
vX

tensor([1.0000, 1.1429, 1.2857, 1.4286, 1.5714, 1.7143, 1.8571, 2.0000, 2.1429,
        2.2857, 2.4286, 2.5714, 2.7143, 2.8571, 3.0000])

In [8]:
vX.numpy()

array([1.       , 1.1428572, 1.2857143, 1.4285715, 1.5714285, 1.7142857,
       1.8571429, 2.       , 2.142857 , 2.2857141, 2.4285715, 2.5714285,
       2.7142856, 2.857143 , 3.       ], dtype=float32)

#### Notice the difference between the following two cells:
(be careful when initialize a tensor with round numbers)

In [9]:
vX = torch.tensor([1, 2, 5, 6.])
vX, vX.type()

(tensor([1., 2., 5., 6.]), 'torch.FloatTensor')

In [10]:
vX = torch.tensor([1, 2, 5, 6])
vX, vX.type()

(tensor([1, 2, 5, 6]), 'torch.LongTensor')

###  Autograd
Consider the following function:
$$y=f\left(x\right)=x^{2}+3$$
$$\implies f'\left(x\right)=2x$$

In [11]:
f = lambda x: x**2 + 3
x = torch.tensor(7., requires_grad=True)
y = f(x)

In [12]:
#-- compute gradients:
y.backward()

In [13]:
#-- check that f'(7) = 14:
x.grad

tensor(14.)

Consider now:
$$y=f\left(\boldsymbol{x},\boldsymbol{w}\right)=\boldsymbol{w}^{T}\boldsymbol{x}$$
$$\implies\nabla_{\boldsymbol{x}}f=\boldsymbol{w}$$
and
$$\implies\nabla_{\boldsymbol{w}}f=\boldsymbol{x}$$

In [14]:
f  = lambda vX, vW: vW[None,:] @ vX[:,None]
vX = torch.tensor([1., 3], requires_grad=True)
vW = torch.tensor([2., 5], requires_grad=True)
y  = f(vX, vW)

In [15]:
#-- compute gradients:
y.backward()

In [16]:
#-- check that:
#--     1. ∇xf = w
#--     2. ∇wf = x
print(vX.grad)
print(vW.grad)

tensor([2., 5.])
tensor([1., 3.])


### Why do we need to set to zero the gradients?
Let us repeat the code from the cells above:

In [17]:
y = f(vX, vW)
y.backward()

In [18]:
print(vX.grad)
print(vW.grad)

tensor([ 4., 10.])
tensor([2., 6.])


Note that the results are different now.  
This is because we did not reset the gradients.  
Let us try again:

In [19]:
vX.grad.data.zero_()
vW.grad.data.zero_()

y = f(vX, vW)
y.backward()

In [20]:
print(vX.grad)
print(vW.grad)

tensor([2., 5.])
tensor([1., 3.])


Consider:
$$\boldsymbol{y}=f\left(\boldsymbol{x}\right)=\boldsymbol{W}\boldsymbol{x}$$
$$\implies\nabla_{\boldsymbol{x}}f\left[\boldsymbol{h}\right]=\boldsymbol{W}\boldsymbol{h}$$
$$\implies\nabla_{\boldsymbol{x}}f=\boldsymbol{W}$$

Since $\boldsymbol{y}$ is a vector, applying:
$$\boldsymbol{y}\text{.backward}\left(\boldsymbol{h}\right)$$
results in $\boldsymbol{h}^{T}\boldsymbol{\nabla}\boldsymbol{y}$  
(or $\boldsymbol{h}^{T}\boldsymbol{\nabla}f$)

In [21]:
f  = lambda mW, vX: mW @ vX
vX = torch.tensor([1., 4], requires_grad=True)
mW = torch.tensor([[1., 4],
                   [2,  1]], requires_grad=True)

vY = f(mW, vX)
vH = torch.ones(2)
vY.backward(vH)

In [22]:
#-- check that:
#--     x.grad = h^T @ ∇xf
print(vX.grad)
print(vH.T @ mW)

tensor([3., 5.])
tensor([3., 5.], grad_fn=<SqueezeBackward3>)


Some imports:

In [23]:
import torch.nn            as nn
import torch.nn.functional as F
import torchsummary

### Sequential model:
$$\hat{\boldsymbol{y}}=f\left(\boldsymbol{x}\right)=\boldsymbol{W}_{3}\sigma\left(\boldsymbol{W}_{2}\sigma\left(\boldsymbol{W}_{1}\boldsymbol{x}+\boldsymbol{b}_{1}\right)+\boldsymbol{b}_{2}\right)$$

In [24]:
oModel = nn.Sequential(
    nn.Linear(100, 50), nn.ReLU(), #-- z1 = σ(W1 * x + b1)
    nn.Linear(50,  25), nn.ReLU(), #-- z2 = σ(W2 * z1 + b2)
    nn.Linear(25,  10, bias=False) #-- y  = W3 * z2
)

torchsummary.summary(oModel, (100,)); print()

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 50]                  5,050
├─ReLU: 1-2                              [-1, 50]                  --
├─Linear: 1-3                            [-1, 25]                  1,275
├─ReLU: 1-4                              [-1, 25]                  --
├─Linear: 1-5                            [-1, 10]                  250
Total params: 6,575
Trainable params: 6,575
Non-trainable params: 0
Total mult-adds (M): 0.01
------------------------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.03
Estimated Total Size (MB): 0.03
------------------------------------------------------------------------------------------



### Custom module (layer)
Consider the following architecture:
$$\hat{\boldsymbol{y}}=f\left(\boldsymbol{x}\right)=\boldsymbol{W}_{3}\left(\sigma_{1}\left(\boldsymbol{W}_{1}\boldsymbol{x}\right)+\sigma_{2}\left(\boldsymbol{W}_{2}\boldsymbol{x}\right)\right)$$

<center> <img src="https://github.com/FixelAlgorithmsTeam/FixelCourses/blob/master/DeepLearningMethods/07_PyTorch/ParallelNetwork.png?raw=true" alt="a" style="width: 500px;"/> </center>

since we compute $\sigma_{1}\left(\boldsymbol{W}_{1}\boldsymbol{x}\right)$ and $\sigma_{2}\left(\boldsymbol{W}_{2}\boldsymbol{x}\right)$ in parallel  
this model can not be implemented using sequential model.

<center> <img src="https://github.com/FixelAlgorithmsTeam/FixelCourses/blob/master/DeepLearningMethods/07_PyTorch/ParallelNetwork.png?raw=true" alt="a" style="width: 200px;"/> </center>
$$\hat{\boldsymbol{y}}=f\left(\boldsymbol{x}\right)=\boldsymbol{W}_{3}\left(\sigma_{1}\left(\boldsymbol{W}_{1}\boldsymbol{x}\right)+\sigma_{2}\left(\boldsymbol{W}_{2}\boldsymbol{x}\right)\right)$$

#### Option I: Define a new (custom) layer:
$$\text{NewLayer}\left(\boldsymbol{x}\right)=\sigma_{1}\left(\boldsymbol{W}_{1}\boldsymbol{x}\right)+\sigma_{2}\left(\boldsymbol{W}_{2}\boldsymbol{x}\right)$$
and then use `nn.Sequential`

In [25]:
class NewLayer(nn.Module):
    def __init__(self, dIn, dOut):
        super(NewLayer, self).__init__() #-- always do this
        self.Linear1 = nn.Linear(dIn, dOut, bias=False)
        self.Linear2 = nn.Linear(dIn, dOut, bias=False)

    def forward(self, mX):
        mZ1 = torch.sigmoid(self.Linear1(mX)) #-- σ1(W1 * x)
        mZ2 = torch.tanh(self.Linear2(mX))    #-- σ2(W2 * x)
        return mZ1 + mZ2

In [26]:
oModel = nn.Sequential(
    NewLayer(100, 50),            #-- z = σ1(W1 * x) + σ2(W2 * x)
    nn.Linear(50, 10, bias=False) #-- y = W3 * z
)
torchsummary.summary(oModel, (100,)); print()

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─NewLayer: 1-1                          [-1, 50]                  --
|    └─Linear: 2-1                       [-1, 50]                  5,000
|    └─Linear: 2-2                       [-1, 50]                  5,000
├─Linear: 1-2                            [-1, 10]                  500
Total params: 10,500
Trainable params: 10,500
Non-trainable params: 0
Total mult-adds (M): 0.02
------------------------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.04
Estimated Total Size (MB): 0.04
------------------------------------------------------------------------------------------



$$\hat{\boldsymbol{y}}=f\left(\boldsymbol{x}\right)=\boldsymbol{W}_{3}\left(\sigma_{1}\left(\boldsymbol{W}_{1}\boldsymbol{x}\right)+\sigma_{2}\left(\boldsymbol{W}_{2}\boldsymbol{x}\right)\right)$$
#### Option II: Manually define the architecture:

In [27]:
class ParallelModel(nn.Module):
    def __init__(self, dIn, dHidden, dOut):
        super(ParallelModel, self).__init__() #-- always do this
        self.Linear1 = nn.Linear(dIn,     dHidden, bias=False)
        self.Linear2 = nn.Linear(dIn,     dHidden, bias=False)
        self.Linear3 = nn.Linear(dHidden, dOut,    bias=False)

    def forward(self, mX):
        mZ1 = torch.sigmoid(self.Linear1(mX)) #-- σ1(W1 * x)
        mZ2 = torch.tanh(self.Linear2(mX))    #-- σ2(W2 * x)
        mY  = self.Linear3(mZ1 + mZ2)         #-- W3 * (σ1(W1 * x) + σ2(W2 * x))
        return mY

In [28]:
oModel = ParallelModel(100, 50, 10)

torchsummary.summary(oModel, (100,)); print()

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 50]                  5,000
├─Linear: 1-2                            [-1, 50]                  5,000
├─Linear: 1-3                            [-1, 10]                  500
Total params: 10,500
Trainable params: 10,500
Non-trainable params: 0
Total mult-adds (M): 0.01
------------------------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.04
Estimated Total Size (MB): 0.04
------------------------------------------------------------------------------------------



### Using GPU
To move data to the GPU we use `.cuda()`, or `.to.('cuda')`

In [29]:
mX = torch.randn(2, 4).cuda()
mX

tensor([[ 1.3708, -2.2602,  0.6343,  0.9463],
        [-0.4742,  1.5674, -1.5246,  0.5486]], device='cuda:0')

In [30]:
mX = torch.randn(2, 4).to('cuda')
mX

tensor([[-8.1211e-04,  1.2354e+00, -4.9935e-01,  1.7026e-01],
        [-9.4099e-01, -6.5428e-01, -4.9636e-01,  1.9003e+00]], device='cuda:0')

In [31]:
#-- Generate data directly inside the GPU
mX = torch.randn(2, 4, device='cuda')
mX

tensor([[ 0.9882, -0.5427,  0.3424, -0.2295],
        [ 0.0432,  0.7909, -1.9155, -0.5075]], device='cuda:0')

In [32]:
#-- Move the parameters of the model to the GPU:
oModel.to('cuda')
next(oModel.parameters()).device

device(type='cuda', index=0)

Back to cpu:

In [33]:
mX = mX.cpu()
#-- or:
mX = mX.to('cpu')
mX.device

device(type='cpu')

###  The End